<a href="https://www.kaggle.com/code/cbeaud/smart-home-s-temperature-evalml?scriptVersionId=108153467" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 style="color: #021f46"> Smart Home's Temperature - Time Series Forecasting </h1>

## Motivation
This notebook is my contribution to the Smart Home's Temperature - Time Series Forecasting [competition](https://www.kaggle.com/competitions/smart-homes-temperature-time-series-forecasting/overview).
I participate in this competition to improve my level in the field of Time Series Forecasting.

## Methodology
We will experiment with the performance of [EvalML](https://evalml.alteryx.com/en/stable/index.html) (an Automated Machine Learning (AutoML) Search algorithm) on this dataset.


## What you earn
After reading this notebook :
- You will be how to do Feature Engineering with Time Series data.
- You will know how to use EvalML for regression tasks.
- And much more...

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Load Data

In [2]:
train=pd.read_csv('../input/smart-homes-temperature-time-series-forecasting/train.csv')
test=pd.read_csv('../input/smart-homes-temperature-time-series-forecasting/test.csv')

train.shape, test.shape

((2764, 19), (1373, 18))

# Understand Data

In [3]:
train.head()

,Id,Date,Time,CO2_(dinning-room),CO2_room,Relative_humidity_(dinning-room),Relative_humidity_room,Lighting_(dinning-room),Lighting_room,Meteo_Rain,Meteo_Sun_dusk,Meteo_Wind,Meteo_Sun_light_in_west_facade,Meteo_Sun_light_in_east_facade,Meteo_Sun_light_in_south_facade,Meteo_Sun_irradiance,Outdoor_relative_humidity_Sensor,Day_of_the_week,Indoor_temperature_room
0,0,13/03/2012,11:45,216.560,221.920,39.9125,42.4150,81.6650,113.520,0.0,623.360,1.42625,9690.24,12604.20,95436.8,758.880,48.3750,2.0,17.8275
1,1,13/03/2012,12:00,219.947,220.363,39.9267,42.2453,81.7413,113.605,0.0,623.211,1.59200,11022.00,10787.20,95436.8,762.069,47.8080,2.0,18.1207
2,2,13/03/2012,12:15,219.403,218.933,39.7720,42.2267,81.4240,113.600,0.0,622.656,1.89133,13960.50,9669.63,95398.6,766.251,47.4320,2.0,18.4367
3,3,13/03/2012,12:30,218.613,217.045,39.7760,42.0987,81.5013,113.344,0.0,622.571,1.82800,18511.20,9648.13,95360.3,766.037,47.0240,2.0,18.7513
4,4,13/03/2012,12:45,217.714,216.080,39.7757,42.0686,81.4657,113.034,0.0,622.400,2.36071,26349.00,9208.32,95354.9,762.743,45.4743,2.0,19.0414


In [4]:
test.head()

,Id,Date,Time,CO2_(dinning-room),CO2_room,Relative_humidity_(dinning-room),Relative_humidity_room,Lighting_(dinning-room),Lighting_room,Meteo_Rain,Meteo_Sun_dusk,Meteo_Wind,Meteo_Sun_light_in_west_facade,Meteo_Sun_light_in_east_facade,Meteo_Sun_light_in_south_facade,Meteo_Sun_irradiance,Outdoor_relative_humidity_Sensor,Day_of_the_week
0,2764,18/04/2012,00:00,207.840,206.560,31.5800,30.1600,11.7200,14.1600,0.0,0.65,0.000000,0.0,0.0,0.0,-3.12000,39.0800,3.0
1,2765,18/04/2012,00:15,207.499,206.101,31.6240,30.0613,11.5240,14.7600,0.0,0.65,0.000000,0.0,0.0,0.0,-2.80800,38.9680,3.0
2,2766,18/04/2012,00:30,207.360,206.805,31.6507,30.1773,11.6220,14.9547,0.0,0.65,0.088000,0.0,0.0,0.0,-2.96400,38.9467,3.0
3,2767,18/04/2012,00:45,207.445,206.325,31.6987,30.2560,11.7847,15.0520,0.0,0.65,0.127333,0.0,0.0,0.0,-3.02933,39.0760,3.0
4,2768,18/04/2012,01:00,207.840,205.952,31.7707,30.3840,11.5887,15.2147,0.0,0.65,0.000000,0.0,0.0,0.0,-2.88600,40.0413,3.0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Id                                2764 non-null   int64  
 1   Date                              2764 non-null   object 
 2   Time                              2764 non-null   object 
 3   CO2_(dinning-room)                2764 non-null   float64
 4   CO2_room                          2764 non-null   float64
 5   Relative_humidity_(dinning-room)  2764 non-null   float64
 6   Relative_humidity_room            2764 non-null   float64
 7   Lighting_(dinning-room)           2764 non-null   float64
 8   Lighting_room                     2764 non-null   float64
 9   Meteo_Rain                        2764 non-null   float64
 10  Meteo_Sun_dusk                    2764 non-null   float64
 11  Meteo_Wind                        2764 non-null   float64
 12  Meteo_

# Feature Engineering

FE credit: @gauravduttakiit [work](https://www.kaggle.com/code/gauravduttakiit/smart-home-s-temperature-flaml-mae)

In [6]:
!pip install fast_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 KB 175.8 kB/s eta 0:00:00


In [7]:
from fast_ml.feature_engineering import FeatureEngineering_DateTime
dt_fe = FeatureEngineering_DateTime()

def features_eng(data):
    # Step 1
    data['Date']=data['Date'].copy() + ' ' + data['Time']
    data['Date']=pd.to_datetime(data['Date'])
    data=data.drop(['Time'],axis=1)
    
    # Step 2
    dt_fe.fit(data, datetime_variables=['Date'])
    data = dt_fe.transform(data)
    
    return data

In [8]:
train = features_eng(train)
test = features_eng(test)

/opt/conda/lib/python3.7/site-packages/fast_ml/feature_engineering.py:382: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[pfx+f] = getattr(df[var].dt, f)


# Data Clean

Copy and edit from : @gauravduttakiit [work](https://www.kaggle.com/code/gauravduttakiit/smart-home-s-temperature-flaml-mae)

In [9]:
nunique_train=train.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
remove_col

['Id',
 'Date',
 'Date:year',
 'Date:is_month_start',
 'Date:is_quarter_start',
 'Date:is_year_end',
 'Date:is_year_start',
 'Date:second']

In [10]:
def data_clean(data):
    # Step 1
    data=data.drop(remove_col,axis=1)
    
    # Step 2 (Data preprocessing)
    data['Date:is_weekend']=data['Date:is_weekend'].replace({False: 0, True: 1})
    data['Date:is_quarter_end']=data['Date:is_quarter_end'].replace({False: 0, True: 1})
    data['Date:is_month_end']=data['Date:is_month_end'].replace({False: 0, True: 1})
    
    # Step 3
    data=data.drop(['Date:time'],axis=1)
    
    # Step 4
    data['Date:day_part'] = data['Date:day_part'].fillna(value=np.nan)
    
    # Step 5
    data['Date:day_part'] = data['Date:day_part'].fillna(value='midnight')
    
    # Step 6
    data['Date:day_part'] = data['Date:day_part'].replace({'midnight':0, 'dawn':1, 'early morning':2, 
                                                               'late morning':3, 'noon':4, 'afternoon':5,
                                                               'evening':6, 'night':7})
    
    return data

In [11]:
train = data_clean(train)
test = data_clean(test)

Function copy and edit from : @gauravduttakiit [work](https://www.kaggle.com/code/gauravduttakiit/smart-home-s-temperature-flaml-mae)

In [12]:
import re

train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

train.head()

,CO2_dinningroom,CO2_room,Relative_humidity_dinningroom,Relative_humidity_room,Lighting_dinningroom,Lighting_room,Meteo_Rain,Meteo_Sun_dusk,Meteo_Wind,Meteo_Sun_light_in_west_facade,...,Dateday,Dateday_of_week,Dateday_of_year,Dateweekofyear,Dateis_month_end,Dateis_quarter_end,Datehour,Dateminute,Dateis_weekend,Dateday_part
0,216.560,221.920,39.9125,42.4150,81.6650,113.520,0.0,623.360,1.42625,9690.24,...,13,1,73,11,0,0,11,45,0,4
1,219.947,220.363,39.9267,42.2453,81.7413,113.605,0.0,623.211,1.59200,11022.00,...,13,1,73,11,0,0,12,0,0,4
2,219.403,218.933,39.7720,42.2267,81.4240,113.600,0.0,622.656,1.89133,13960.50,...,13,1,73,11,0,0,12,15,0,4
3,218.613,217.045,39.7760,42.0987,81.5013,113.344,0.0,622.571,1.82800,18511.20,...,13,1,73,11,0,0,12,30,0,4
4,217.714,216.080,39.7757,42.0686,81.4657,113.034,0.0,622.400,2.36071,26349.00,...,13,1,73,11,0,0,12,45,0,4


# Split Data

In [13]:
X = train.copy()
y = X.pop('Indoor_temperature_room')

In [14]:
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

# Modeling with EvalML

EvalML [docs here](https://evalml.alteryx.com/en/stable/user_guide/automl.html)

- Installation

In [15]:
!python3 -m pip install -q evalml==0.28.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
tfx-bsl 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tfx-bsl 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<3,>=1.15.5, but you have tensorflow 2.6.3 which is incompatible.
tensorflow-transform 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tensorflow-transform 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.3 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.7 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.5.2 which is incompatible.
mxnet 1.9.0 requires gra

In [16]:
from evalml.automl import AutoMLSearch

### Training

In [17]:
# run model
model_evalml = AutoMLSearch(X_train=X, y_train=y, problem_type='regression', max_time=1200) # 20 minutes
model_evalml.search()

Generating pipelines to search over...


8 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Will stop searching for new pipelines after 1200 seconds.

Allowed model families: xgboost, lightgbm, random_forest, linear_model, catboost, decision_tree, extra_trees



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean R2: -0.000
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Mean Baseline Regression Pipeline may not perform as estimated on unseen data.

*****************************
* Evaluating Batch Number 1 *
*****************************

Linear Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.751
Decision Tree Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.857
Random Forest Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.927
LightGBM Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.933


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.801e+03, tolerance: 1.481e+00

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.761e+03, tolerance: 1.481e+00



Elastic Net Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.742


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.767e+03, tolerance: 1.436e+00



XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.987
Extra Trees Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.915
CatBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.336

*****************************
* Evaluating Batch Number 2 *
*****************************

XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.951
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.979
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.949
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.951
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.990

*****************************
* Evaluating Batch Number 3 *
*****************************

Lig

### Leaderboard

In [18]:
# check leaderboard
model_evalml.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,51,XGBoost Regressor w/ Imputer,51,0.990865,0.000286,0.990661,213014.020953,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,55,LightGBM Regressor w/ Imputer,55,0.987985,0.001025,0.987290,212395.140111,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,168,CatBoost Regressor w/ Imputer,168,0.986742,0.000353,0.986974,212128.132262,False,{'Imputer': {'categorical_impute_strategy': 'm...
19,100,Random Forest Regressor w/ Imputer,100,0.984225,0.002860,0.984149,211587.325917,False,{'Imputer': {'categorical_impute_strategy': 'm...
73,104,Extra Trees Regressor w/ Imputer,104,0.948602,0.005557,0.954687,203932.741121,False,{'Imputer': {'categorical_impute_strategy': 'm...
79,149,Decision Tree Regressor w/ Imputer,149,0.938524,0.016089,0.944325,201767.184019,False,{'Imputer': {'categorical_impute_strategy': 'm...
120,38,Linear Regressor w/ Imputer + Standard Scaler,38,0.750882,0.007047,0.755113,161447.243636,False,{'Imputer': {'categorical_impute_strategy': 'm...
142,5,Elastic Net Regressor w/ Imputer + Standard Sc...,5,0.741675,0.008865,0.744859,159468.871192,False,{'Imputer': {'categorical_impute_strategy': 'm...
174,0,Mean Baseline Regression Pipeline,0,-0.000465,0.000410,-0.000033,0.000000,True,{'Baseline Regressor': {'strategy': 'mean'}}


### Feature Importance

### Evaluate Model

In [19]:
import math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluates_model(y_test, y_pred):
    print("*"*12, "Evaluations", "*"*12, '\n')
    
    print("MAE model :", mean_absolute_error(y_test, y_pred))
    
    print("MSE model :", mean_squared_error(y_test, y_pred))
    
    print("R2_Score model :", r2_score(y_test, y_pred))
    
    mse_1 = np.square(np.subtract(y_test,y_pred)).mean() 
    
    print("RMSE model :", math.sqrt(mse_1))

In [20]:
model_evalml.best_pipeline

pipeline = RegressionPipeline(component_graph={'Imputer': ['Imputer'], 'XGBoost Regressor': ['XGBoost Regressor', 'Imputer.x']}, parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'XGBoost Regressor':{'eta': 0.03842638804730825, 'max_depth': 12, 'min_child_weight': 9.895364542533038, 'n_estimators': 965, 'n_jobs': -1}}, random_seed=0)

In [21]:
pred = model_evalml.best_pipeline.predict(X_test)
evaluates_model(y_test, pred)

************ Evaluations ************ 

MAE model : 0.009103630630309355
MSE model : 0.00020197414423661712
R2_Score model : 0.9999739388598132
RMSE model : 4.448307648548899


# Make a Test prediction


In [22]:
## generate predictions
y_predict = model_evalml.best_pipeline.predict(test)

len(y_predict)

1373

# Submission

In [23]:
sub=pd.read_csv('../input/smart-homes-temperature-time-series-forecasting/sample_submission.csv')
print(sub.shape)
sub.head()

(1373, 2)


,Id,Indoor_temperature_room
0,2764,0.0
1,2765,0.0
2,2766,0.0
3,2767,0.0
4,2768,0.0


In [24]:
!pip install pandas -U
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 11.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
woodwork 0.4.2 requires pandas<1.2.5,>=1.2.0, but you have pandas 1.3.5 which is incompatible.
tfx-bsl 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tfx-bsl 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<3,>=1.15.5, but you have tensorflow 2.6.3 which is incompatible.
tensorflow-transform 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tensorflow-transform 1.7.0 requires tensorflow!=2.0.*,!=

In [25]:
sub['Indoor_temperature_room']=y_predict
sub.to_csv('submission.csv', index=False)
sub.head()

,Id,Indoor_temperature_room
0,2764,19.499954
1,2765,19.477037
2,2766,19.423576
3,2767,19.323776
4,2768,19.300669


# What Next ?

- Try to use a EvalML [Time Serie solution](https://evalml.alteryx.com/en/stable/user_guide/timeseries.html#)
- Experiment with other Feature Engineering
- Experiment with other AutoML algorithms (like H20, MLJAR)

# Ressources

- Feature Engineering, Data Clean: @gauravduttakiit [work](https://www.kaggle.com/code/gauravduttakiit/smart-home-s-temperature-flaml-mae)
- EvalML documentation : https://evalml.alteryx.com/en/stable/index.html

<center>
    <h1 style="color:#021f46"> I hope you learned something new today 🙃 
        Thanks for reading  </h1>